In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from tinyshift.series import adi_cv, theoretical_limit, foreca, stability_index, hurst_exponent, trend_significance
from utilsforecast.preprocessing import fill_gaps
from utilsforecast.data import generate_series
from statsmodels.tsa.stattools import adfuller
from scipy import signal

In [2]:
df = generate_series(3, with_trend=True, static_as_categorical=False, seed=42, freq="D")

In [3]:
df.isnull().sum()

unique_id    0
ds           0
y            0
dtype: int64

In [4]:
df = fill_gaps(df, freq="D", end="per_serie", id_col="unique_id", time_col="ds")

In [5]:
def remove_leading_zeros(group):
    """
    Removes leading zeros from series
    """
    first_non_zero_index = group["y"].ne(0).idxmax()
    return group.loc[first_non_zero_index:]

In [6]:
df = df.groupby("unique_id")[df.columns].apply(remove_leading_zeros).reset_index(drop=True)

In [7]:
def is_obsolete(group, days_obsoletes):
    """
    Identify obsolote series
    """
    last_date = group["ds"].max()
    cutoff_date = last_date - pd.Timedelta(days=days_obsoletes)
    recent_data = group[group["ds"] >= cutoff_date]
    return (recent_data["y"] ==0).all()

In [8]:
days_obsoletes=180
obsolete_series = df.groupby("unique_id")[df.columns].apply(is_obsolete, days_obsoletes)

In [9]:
obsolote_ids = obsolete_series[obsolete_series].index.tolist()

In [10]:
obsolote_ids

[]

In [11]:
for id in df["unique_id"].unique():
    print(id, trend_significance(df.loc[df["unique_id"] == id, "y"]))

0 (np.float64(0.9894087777029937), np.float64(4.8574508252298796e-150))
1 (np.float64(0.998759437594496), np.float64(0.0))
2 (np.float64(0.9990776434701963), np.float64(0.0))


In [12]:
px.line(df, x="ds", y="y", facet_col="unique_id")

In [13]:
px.line(df, x="ds", y="y", facet_col="unique_id")

In [14]:
def forecastability(X):
    """
    Calculate forecastability metrics for a time series.
    """
    return {"foreCA": foreca(X),
     "theoretical_limit": theoretical_limit(X),
     "stability_index": stability_index(X, detrend=True),
     "adi_cv": adi_cv(X),
     "hurst_exponent": hurst_exponent(X)}

In [15]:
for id in df["unique_id"].unique():
    print(id, forecastability(df.loc[df["unique_id"] == id, "y"]))

0 {'foreCA': 0.6167336082009967, 'theoretical_limit': np.float64(0.5607476664580777), 'stability_index': np.float64(0.9379217273954117), 'adi_cv': (1.0, np.float64(0.28499520299004516)), 'hurst_exponent': (0.3290861603798452, 0.13218639190520123)}
1 {'foreCA': 0.7024148354847279, 'theoretical_limit': np.float64(0.5555700115725419), 'stability_index': np.float64(0.9567001199267816), 'adi_cv': (1.0, np.float64(0.31390442751597913)), 'hurst_exponent': (0.28893923895853546, 0.032089718874560705)}
2 {'foreCA': 0.6917944469563875, 'theoretical_limit': np.float64(0.5581737595146441), 'stability_index': np.float64(0.959329022584575), 'adi_cv': (1.0, np.float64(0.31714749237101025)), 'hurst_exponent': (0.2801838603106017, 0.028603261603533478)}


# ADF Test - Order of Diff 1

In [16]:
for id in df["unique_id"].unique():
    print(id, adfuller(np.diff(df.loc[df["unique_id"] == id, "y"])))

0 (np.float64(-7.7297494388459045), np.float64(1.1336858770700725e-11), 12, 138, {'1%': np.float64(-3.47864788917503), '5%': np.float64(-2.882721765644168), '10%': np.float64(-2.578065326612056)}, np.float64(-66.72626559222371))
1 (np.float64(-15.384231114998379), np.float64(3.3758074678499965e-28), 12, 471, {'1%': np.float64(-3.444310291143469), '5%': np.float64(-2.867695986993458), '10%': np.float64(-2.5700489042602586)}, np.float64(-272.6047974446162))
2 (np.float64(-11.210908977921001), np.float64(2.1267705530415887e-20), 13, 383, {'1%': np.float64(-3.44753973676872), '5%': np.float64(-2.8691160516676844), '10%': np.float64(-2.57080585953957)}, np.float64(-229.38230776890555))


# Domingo

- #TODO: Adicionar gráfico de ACF
- #TODO: Adicionar gráfico de PACF
- #TODO: Interpretar
- #TODO: Adicionar gráficos da aula do valeriy
- #TODO: Comparativo de dados diferenciados
- #TODO: Trazer gráficos de ipynbs dele